In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [7]:
r = requests.get("https://api-pc6dbtrtla-uc.a.run.app/API/timeseries/usa")
response_dict = r.json()
dataset = pd.DataFrame.from_dict(response_dict)
dataset = dataset.rename(columns={'Total Results as of Date': 'Date'})
Days = (np.array([i for i in range (len(dataset.Date))]).reshape(-1,1))+1
dataset['Days'] = Days
dataset['USPop'] = 328200000
dataset['Infection%'] = dataset['Cases'] / dataset['USPop'] * 100
dataset['Recovered%'] = dataset['Recovered'] / dataset['USPop'] * 100
dataset['Deaths%'] = dataset['Deaths'] / dataset['USPop'] * 100
dataset['Recovered%ofInfected'] = dataset['Recovered%'] / dataset['Infection%'] * 100
dataset['dead%ofInfected'] = dataset['Deaths%'] / dataset['Infection%'] * 100

# create lag calculating date before what happen. Numerical Change based off previous days data
#ADD variable that is deaths yesterday, let it be the deaths from the day before
#Could add a lag of 2/5/10/ whatever (Perhaps set lag to 14+ days to account for incubation period)
#This is panel data, or works like it
#Start somewhere meaningful, research a good start date
#Try using linear regression with TimeSeriesSplit for splits
#Need to know how long the incubation period is/how long the recovery time is/How long people go from initial symptoms to dying
#dataset.loc[dataset['Days'] == 104]

dataset.tail(60)

,ISO3,Country,Date,Cases,Deaths,Recovered,Days,USPop,Infection%,Recovered%,Deaths%,Recovered%ofInfected,dead%ofInfected
46,USA,US,2020-03-08,518.0,21.0,7.0,47,328200000,0.000158,0.000002,0.000006,1.351351,4.054054
47,USA,US,2020-03-09,583.0,22.0,7.0,48,328200000,0.000178,0.000002,0.000007,1.200686,3.773585
48,USA,US,2020-03-10,959.0,28.0,8.0,49,328200000,0.000292,0.000002,0.000009,0.834202,2.919708
49,USA,US,2020-03-11,1281.0,36.0,8.0,50,328200000,0.000390,0.000002,0.000011,0.624512,2.810304
50,USA,US,2020-03-12,1663.0,41.0,12.0,51,328200000,0.000507,0.000004,0.000012,0.721587,2.465424
51,USA,US,2020-03-13,2179.0,49.0,12.0,52,328200000,0.000664,0.000004,0.000015,0.550711,2.248738
52,USA,US,2020-03-14,2727.0,58.0,12.0,53,328200000,0.000831,0.000004,0.000018,0.440044,2.126879
53,USA,US,2020-03-15,3499.0,73.0,12.0,54,328200000,0.001066,0.000004,0.000022,0.342955,2.086310
54,USA,US,2020-03-16,4632.0,99.0,17.0,55,328200000,0.001411,0.000005,0.000030,0.367012,2.137306
55,USA,US,2020-03-17,6421.0,133.0,17.0,56,328200000,0.001956,0.000005,0.000041,0.264756,2.071328


Linear model will have a non-linear effect. In days it will not be linear
Facebook

In [4]:
X = dataset[["Days"]]
y = dataset["dead%ofInfected"]

In [5]:
y

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
101    5.860196
102    5.844530
103    5.838992
104    5.900605
105    5.976788
Name: dead%ofInfected, Length: 106, dtype: float64

In [5]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 42)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [7]:
## Predicting the Test set results

y_pred = regressor.predict(X_test)

# y_pred = regressor.predict([[105]])
# y_pred

In [16]:
regressor.coef_

AttributeError: 'RandomForestRegressor' object has no attribute 'coef_'

In [8]:
## Evaluating the Model Performance - R2 Score
r2_score(y_test, y_pred)


0.9679030276829057

In [9]:
## Evaluating the Model Performance - Mean-squared Error
mean_squared_error(y_test, y_pred)

0.19198395688043102

In [10]:
## Applying k-Fold Cross Validation
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 93.89 %
Standard Deviation: 8.68 %


In [11]:
y_pred

array([5.88481015, 0.        , 0.        , 4.72846664, 5.72313116,
       2.09956031, 0.        , 0.        , 5.70279345, 0.        ,
       3.94061945, 5.39366829, 0.        , 4.02835868, 0.        ,
       2.00047416, 2.53783387, 4.37609836, 2.103138  , 5.10466412,
       0.        , 4.72846664])

In [12]:
X_test

,Days
100,101
10,11
4,5
81,82
97,98
65,66
30,31
33,34
93,94
11,12


In [13]:
dataset['Cases'][81]

555313.0

In [14]:
dataset['Deaths'][81]

26086.0

In [15]:
dataset['Cases'][81] * .0454560198

25242.318723197397